<a href="https://colab.research.google.com/github/King-Rian/Project-3/blob/main/Project_3_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Install emoji library
!pip install emoji
!pip install torch
from google.colab import files
import zipfile
import json
import emoji
import pandas as pd
import torch
import re
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaTokenizer, RobertaModel, AdamW, T5Tokenizer, T5ForConditionalGeneration

In [36]:
# Retrieve the 'emojis.json' zippped file

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Extract the ZIP file
with zipfile.ZipFile('emojis.json.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Step 3: Load the extracted JSON file
emoji_file = 'emojis.json'

# Using pandas
emoji_df = pd.read_json(emoji_file)
print("Dataset loaded successfully. Sample data:")
print(emoji_df.head())


Saving emojis.json.zip to emojis.json (1).zip
Dataset loaded successfully. Sample data:
                                            category  \
0  Miscellaneous Symbols And Pictographs -> Emoji...   
1  Miscellaneous Symbols And Pictographs -> Emoji...   
2                                               None   
3  Miscellaneous Symbols And Pictographs -> Emoji...   
4                                               None   

                                            keywords  \
0  [dark skin tone, hand, forbidden, gesture, wom...   
1                     [dark skin tone, woman, guard]   
2                 [racing, running, woman, marathon]   
3  [gymnastics, medium-light skin tone, woman, ca...   
4                                      [woman, golf]   

                                          definition  \
0  The Woman Gesturing Not OK, Type-6 emoji is a ...   
1  The Female Guard, Type-6 emoji is a sequence o...   
2  The female version of the ?? Runner emoji. The...   
3  The Woman D

In [37]:
# Check the shape, features, and datatypes of the df
emoji_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389 entries, 0 to 2388
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    1988 non-null   object
 1   keywords    2389 non-null   object
 2   definition  2389 non-null   object
 3   unicode     2389 non-null   object
 4   name        2389 non-null   object
 5   shortcode   845 non-null    object
 6   senses      2389 non-null   object
dtypes: object(7)
memory usage: 130.8+ KB


In [38]:
# Set options to display all rows and columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

# Create a new emojis DataFrame with 'name' and 'unicode'
key_value_pairs_df = emoji_df[['name', 'unicode']].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Process the 'name' column to include : at the beginning and end of the emoji name
key_value_pairs_df['name'] = key_value_pairs_df['name'].apply(
    lambda x: f":{x.split(':')[0].strip()}:"  # Remove everything after the first colon and add colons
)

# Convert the unicode column to emoji symbols
key_value_pairs_df['emoji'] = key_value_pairs_df['name'].apply(emoji.emojize)

# Remove rows where 'emoji' contains text with a beginning and ending ':' - which means that emoji was not found for that :name:
filtered_df = key_value_pairs_df[~key_value_pairs_df['emoji'].str.match(r'^:.*:$')].copy()  # Use .copy() here as well

# Remove rows where 'unicode' contains multiple Unicode values (separated by a space)
filtered_df = filtered_df[~filtered_df['unicode'].str.contains(' ', na=False)]

# Create a new df and rename features to read for concatenation with the next data set
new_filtered_df = filtered_df[['name', 'emoji']].copy()  # Use .copy() here as well
new_filtered_df = new_filtered_df.rename(columns={'name': 'label'})

# Display the DataFrame with emoji symbols
new_filtered_df.head()

,label,emoji
15,:squid:,🦑
16,:shrimp:,🦐
17,:rhinoceros:,🦏
18,:lizard:,🦎
19,:gorilla:,🦍


In [39]:
# Filter the DataFrame where the 'label' column contains the word 'car'
car_related_rows = new_filtered_df[new_filtered_df['label'].str.contains('car', case=False, na=False)]

# Display the filtered rows
print(car_related_rows)

       label emoji
54  :carrot:     🥕


In [40]:
# Create an emoji dictionary with labels and their Unicode values from https://carpedm20.github.io/emoji/docs/index.html
emoji_dict = {emoji.demojize(e): e for e in emoji.EMOJI_DATA}

# Show how many key value pairs we retrieved from emoji library
records = len(emoji_dict)
print(f"Number of records: {records}")

# Create new df for use in concatenation the two data frames
emoji_dictionary_df = pd.DataFrame(emoji_dict.items(), columns=['label', 'emoji'])
emoji_dictionary_df.head()
#emoji_dictionary_df.info()

Number of records: 3790


,label,emoji
0,:1st_place_medal:,🥇
1,:2nd_place_medal:,🥈
2,:3rd_place_medal:,🥉
3,:AB_button_(blood_type):,🆎
4,:ATM_sign:,🏧


In [41]:
# Filter the DataFrame where the 'label' column contains the word 'car'
car_related_rows = emoji_dictionary_df[emoji_dictionary_df['label'].str.contains('car', case=False, na=False)]

# Display the filtered rows
print(car_related_rows)

                                              label emoji
56                          :Caribbean_Netherlands:    🇧🇶
179                                    :Madagascar:    🇲🇬
220                                     :Nicaragua:    🇳🇮
565                                 :card_file_box:     🗃
566                                    :card_index:     📇
567                           :card_index_dividers:     🗂
568                                :carousel_horse:     🎠
569                                 :carp_streamer:     🎏
570                                 :carpentry_saw:     🪚
571                                        :carrot:     🥕
780                                   :credit_card:     💳
1057                         :flower_playing_cards:     🎴
1271                          :identification_card:     🪪
1569                             :man_cartwheeling:   🤸‍♂
1570              :man_cartwheeling_dark_skin_tone:  🤸🏿‍♂
1571             :man_cartwheeling_light_skin_tone:  🤸🏻‍♂
1572       :ma

In [42]:
# Concatentate both dataframes / sources and drop the index
combined_emoji_df = pd.concat([emoji_dictionary_df, new_filtered_df], ignore_index=True)

# Show the key pair data frame with training data
combined_emoji_df.head()

,label,emoji
0,:1st_place_medal:,🥇
1,:2nd_place_medal:,🥈
2,:3rd_place_medal:,🥉
3,:AB_button_(blood_type):,🆎
4,:ATM_sign:,🏧


In [43]:
# Make sure 'label' field is lower case, contains no special characters, and has a space between words
# Modify the 'label' column: convert to lowercase, strip spaces, remove non-alphanumeric characters, and replace underscores with spaces
combined_emoji_df['label'] = combined_emoji_df['label'].str.lower().str.strip()

# Use regex to remove non-alphanumeric characters, but keep spaces and replace underscores with spaces
combined_emoji_df['label'] = combined_emoji_df['label'].apply(lambda x: re.sub(r'[^a-z0-9\s_]', '', x))  # Remove non-alphanumeric except _

# Replace underscores with spaces
combined_emoji_df['label'] = combined_emoji_df['label'].apply(lambda x: x.replace('_', ' '))

# Ensure a single space between words (in case there are multiple spaces)
combined_emoji_df['label'] = combined_emoji_df['label'].apply(lambda x: re.sub(r'\s+', ' ', x))

# Display the modified dataframe
print(combined_emoji_df.head())


                  label emoji
0       1st place medal     🥇
1       2nd place medal     🥈
2       3rd place medal     🥉
3  ab button blood type     🆎
4              atm sign     🏧


In [44]:
# Define a function to count the number of words in a string
def count_words(text):
    return len(text.split())

# Apply the function to the 'label' column to filter rows with more than 2 words
combined_emoji_df = combined_emoji_df[combined_emoji_df['label'].apply(count_words) <= 1]

# Display the filtered DataFrame
print(combined_emoji_df)


                 label    emoji
6          afghanistan       🇦🇫
7              albania       🇦🇱
8              algeria       🇩🇿
10             andorra       🇦🇩
11              angola       🇦🇴
12            anguilla       🇦🇮
13          antarctica       🇦🇶
15            aquarius        ♒
16           argentina       🇦🇷
17               aries        ♈
18             armenia       🇦🇲
19               aruba       🇦🇼
21           australia       🇦🇺
22             austria       🇦🇹
23          azerbaijan       🇦🇿
26             bahamas       🇧🇸
27             bahrain       🇧🇭
28          bangladesh       🇧🇩
29            barbados       🇧🇧
30             belarus       🇧🇾
31             belgium       🇧🇪
32              belize       🇧🇿
33               benin       🇧🇯
34             bermuda       🇧🇲
35              bhutan       🇧🇹
36             bolivia       🇧🇴
38            botswana       🇧🇼
40              brazil       🇧🇷
43              brunei       🇧🇳
44            bulgaria       🇧🇬
46      

In [45]:
# Save processed dataset
combined_emoji_df.to_csv('processed_emojis.csv', index=False)

# Display dataset
print(combined_emoji_df.head())

          label emoji
6   afghanistan    🇦🇫
7       albania    🇦🇱
8       algeria    🇩🇿
10      andorra    🇦🇩
11       angola    🇦🇴


In [46]:
# Filter the DataFrame where the 'label' column contains the word 'car'
car_related_rows = combined_emoji_df[combined_emoji_df['label'].str.contains('car', case=False, na=False)]

# Display the filtered rows
print(car_related_rows)



           label emoji
179   madagascar    🇲🇬
220    nicaragua    🇳🇮
571       carrot     🥕
2683     placard     🪧
2875       scarf     🧣
3808      carrot     🥕


In [47]:
# Use simple random oversampling to improve model output

# Count the frequency of each emoji
emoji_counts = combined_emoji_df['emoji'].value_counts()

# Find the emoji with the maximum count
max_count = emoji_counts.max()

# Create an empty list to store the oversampled data
oversampled_data = []

# Iterate through each emoji class and oversample
for emoji, count in emoji_counts.items():
    # Calculate the number of repetitions needed
    n_repeats = max_count - count
    # Get the rows corresponding to this emoji
    emoji_data = combined_emoji_df[combined_emoji_df['emoji'] == emoji]
    # Sample (with replacement) to match the maximum count
    oversampled_data.append(emoji_data.sample(n=n_repeats, replace=True))

# Concatenate the oversampled data back together
oversampled_df = pd.concat(oversampled_data, axis=0)

# Combine the original dataset with the oversampled data
combined_emoji_df = pd.concat([combined_emoji_df, oversampled_df], axis=0)

# Shuffle the final dataset
combined_emoji_df = combined_emoji_df.sample(frac=1).reset_index(drop=True)

print(f"Oversampled dataset shape: {combined_emoji_df.shape}")
combined_emoji_df.head()


Oversampled dataset shape: (2364, 2)


,label,emoji
0,detective,🕵
1,mouse,🐁
2,lipstick,💄
3,paintbrush,🖌️
4,honduras,🇭🇳


In [48]:
# Filter the DataFrame where the 'label' column contains the word 'car'
car_related_rows = combined_emoji_df[combined_emoji_df['label'].str.contains('car', case=False, na=False)]

# Display the filtered rows
print(car_related_rows)

           label emoji
143      placard     🪧
166       carrot     🥕
259   madagascar    🇲🇬
407        scarf     🧣
509      placard     🪧
820       carrot     🥕
894    nicaragua    🇳🇮
1038  madagascar    🇲🇬
1137  madagascar    🇲🇬
1289       scarf     🧣
1404     placard     🪧
1408   nicaragua    🇳🇮
1836   nicaragua    🇳🇮
2063       scarf     🧣
2071      carrot     🥕


In [49]:
import pandas as pd
import nltk
from nltk.corpus import wordnet

# Download the wordnet data
nltk.download('wordnet')

# Function to find synonyms
def get_synonyms(word, limit=2):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.name() not in synonyms:
                synonyms.append(lemma.name())
            if len(synonyms) == limit:
                break
        if len(synonyms) == limit:
            break
    return synonyms

# Create a new DataFrame to store the results
new_data = []

# Loop through each row in the original DataFrame
for index, row in combined_emoji_df.iterrows():
    label = row['label']
    emoji = row['emoji']
    synonyms = get_synonyms(label)

    # For each synonym, create a new record
    for synonym in synonyms:
        if synonym != label:
            new_data.append({'label': synonym, 'emoji': emoji})

# Convert the new data into a DataFrame
df_new = pd.DataFrame(new_data)

# Function to replace underscores with spaces
def replace_underscores(text):
  return text.replace('_', ' ')

# Apply the function to the 'label' column
df_new['label'] = df_new['label'].apply(replace_underscores)

# Display the new DataFrame
print(df_new)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                          label    emoji
0                                  investigator        🕵
1                                        shiner        🐁
2                                     lip rouge        💄
3                                      Honduras       🇭🇳
4                          Republic of Honduras       🇭🇳
5                                        Norway       🇳🇴
6                             Kingdom of Norway       🇳🇴
7                                  customs duty        🛃
8                                      wanderer        🕷
9                                          scab        🐀
10                                   Montenegro       🇲🇪
11                                    Crna Gora       🇲🇪
12                                        Chile       🇨🇱
13                            Republic of Chile       🇨🇱
14                                   Uzbekistan       🇺🇿
15                       Republic of Uzbekistan       🇺🇿
16                             

In [50]:
# Concatenate the two DataFrames
combined_emoji_df = pd.concat([combined_emoji_df, df_new], ignore_index=True)

# Display the concatenated DataFrame
print(combined_emoji_df)


                                          label    emoji
0                                     detective        🕵
1                                         mouse        🐁
2                                      lipstick        💄
3                                    paintbrush       🖌️
4                                      honduras       🇭🇳
5                                        norway       🇳🇴
6                                       customs        🛃
7                                        spider        🕷
8                                           rat        🐀
9                                    montenegro       🇲🇪
10                                        chile       🇨🇱
11                                   fourthirty        🕟
12                                         niue       🇳🇺
13                                   uzbekistan       🇺🇿
14                                       shovel        🪏
15                                     maldives       🇲🇻
16                             

In [51]:
# Apply the function to the 'label' column to filter rows with more than 2 words
combined_emoji_df = combined_emoji_df[combined_emoji_df['label'].apply(count_words) <= 1]

# Display the filtered DataFrame
print(combined_emoji_df)

                   label    emoji
0              detective        🕵
1                  mouse        🐁
2               lipstick        💄
3             paintbrush       🖌️
4               honduras       🇭🇳
5                 norway       🇳🇴
6                customs        🛃
7                 spider        🕷
8                    rat        🐀
9             montenegro       🇲🇪
10                 chile       🇨🇱
11            fourthirty        🕟
12                  niue       🇳🇺
13            uzbekistan       🇺🇿
14                shovel        🪏
15              maldives       🇲🇻
16              motorway        🛣
17                 snail        🐌
18                 olive        🫒
19           firefighter      🧑‍🚒
20               avocado        🥑
21          supervillain        🦹
22                trkiye       🇹🇷
23               glasses        👓
24          technologist      🧑‍💻
25                  herb        🌿
26            basketball        🏀
27               postbox        📮
28            

In [52]:
# Filter the DataFrame where the 'label' column contains the word 'car'
car_related_rows = combined_emoji_df[combined_emoji_df['label'].str.contains('car', case=False, na=False)]

# Display the filtered rows
print(car_related_rows)

           label emoji
143      placard     🪧
166       carrot     🥕
259   madagascar    🇲🇬
407        scarf     🧣
509      placard     🪧
820       carrot     🥕
894    nicaragua    🇳🇮
1038  madagascar    🇲🇬
1137  madagascar    🇲🇬
1289       scarf     🧣
1404     placard     🪧
1408   nicaragua    🇳🇮
1836   nicaragua    🇳🇮
2063       scarf     🧣
2071      carrot     🥕
2385    escargot     🐌
2646  Madagascar    🇲🇬
3115    escargot     🐌
3361   Nicaragua    🇳🇮
3517  Madagascar    🇲🇬
3620  Madagascar    🇲🇬
3692         car     🚗
3803    escargot     🐌
3905   Nicaragua    🇳🇮
4073         car     🚗
4243         car     🚗
4390   Nicaragua    🇳🇮


In [53]:
# Identify rows where 'label' and 'emoji' have the same values
rows_to_remove = combined_emoji_df[combined_emoji_df['label'] == combined_emoji_df['emoji']]

# Remove these rows
combined_emoji_df = combined_emoji_df.drop(rows_to_remove.index)

# Display the updated DataFrame
print("\nUpdated DataFrame:")
print(combined_emoji_df)



Updated DataFrame:
                   label    emoji
0              detective        🕵
1                  mouse        🐁
2               lipstick        💄
3             paintbrush       🖌️
4               honduras       🇭🇳
5                 norway       🇳🇴
6                customs        🛃
7                 spider        🕷
8                    rat        🐀
9             montenegro       🇲🇪
10                 chile       🇨🇱
11            fourthirty        🕟
12                  niue       🇳🇺
13            uzbekistan       🇺🇿
14                shovel        🪏
15              maldives       🇲🇻
16              motorway        🛣
17                 snail        🐌
18                 olive        🫒
19           firefighter      🧑‍🚒
20               avocado        🥑
21          supervillain        🦹
22                trkiye       🇹🇷
23               glasses        👓
24          technologist      🧑‍💻
25                  herb        🌿
26            basketball        🏀
27               postbox    

In [54]:
# Initialize the T5 tokenizer and model
# Load the "t5-small" model and tokenizer from Hugging Face's transformers library
# T5 (Text-to-Text Transfer Transformer) is a model capable of various NLP tasks
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Prepare data for T5
# Convert the 'label' column to a list of input texts and the 'emoji' column to a list of target texts
# These will be the input-output pairs for the model training
input_texts = combined_emoji_df['label'].tolist()  # Use 'label' for model input
target_texts = combined_emoji_df['emoji'].tolist()      # Use 'emoji' for model output

# Tokenize inputs and targets
# Tokenize input texts and target texts for the model, ensuring uniform tensor shapes
# Apply padding and truncation to control the sequence length
# max_length specifies the maximum token length for inputs and outputs
input_encodings = tokenizer(input_texts, padding=True, truncation=True, max_length=32, return_tensors="pt")
target_encodings = tokenizer(target_texts, padding=True, truncation=True, max_length=8, return_tensors="pt")

# Dataset and DataLoader
# Define a custom dataset class for organizing the input and target encodings
class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets):
        # Initialize with tokenized inputs and targets
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        # Return the total number of samples in the dataset
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        # Retrieve a single sample (input, attention mask, and labels) by index
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.targets['input_ids'][idx]
        }

# Create a dataset and data loader
# Use the custom dataset class to organize tokenized inputs and targets
# DataLoader splits the dataset into manageable batches for training
dataset = EmojiDataset(input_encodings, target_encodings)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

# Training setup
# Define an optimizer (AdamW) to update the model's parameters during training
# Set the learning rate (lr) for the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# Set the model to training mode
# This activates certain layers like dropout that are specific to training
model.train()

# Training loop
# Set number of epochs
num_epochs = 3

# Train the model over multiple epochs
for epoch in range(num_epochs):
    total_loss = 0  # Track the total loss for this epoch
    for batch in data_loader:
        # Zero the gradients to prevent accumulation from previous steps
        optimizer.zero_grad()

        # Extract input_ids, attention_mask, and labels from the batch
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Perform a forward pass and compute the loss
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backpropagate the gradients
        loss.backward()

        # Update the model's parameters
        optimizer.step()

        # Accumulate the loss for this batch
        total_loss += loss.item()

    # Print the loss for the current epoch
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Save the model and tokenizer
# This allows reloading the model later for inference or fine-tuning
model.save_pretrained("emoji_t5_model")
tokenizer.save_pretrained("emoji_t5_model")

# Translate input text to emoji
# Define a function to generate emoji translations from text inputs
def translate_to_emoji(text):
    # Set the model to evaluation mode (disables dropout layers)
    model.eval()

    # Tokenize the input text and convert it to a tensor
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=32, truncation=True)

    # Generate output tokens using beam search for better quality translations
    outputs = model.generate(input_ids, max_length=8, num_beams=4, early_stopping=True)

    # Decode the generated tokens to a human-readable string
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the translation
# Provide sample inputs to test the model's ability to generate emoji translations
print(translate_to_emoji("happy"))  # Expected to output an emoji for happiness
print(translate_to_emoji("sad"))    # Expected to output an emoji for sadness
print(translate_to_emoji("love"))   # Expected to output an emoji for love


Epoch 1, Loss: 220.35360337700695
Epoch 2, Loss: 14.07211118657142
Epoch 3, Loss: 11.268302984768525





In [55]:
def translate_to_emoji_with_fallback(text):
    """
    Translates the input text to an emoji using the trained T5 model.
    Falls back to direct mapping if model output is empty or invalid.
    Returns only one emoji for each word entered.
    """
    model.eval()
    try:
        # Generate output with the model
        input_ids = tokenizer.encode(text, return_tensors="pt", max_length=32, truncation=True)
        outputs = model.generate(input_ids, max_length=8, num_beams=4, early_stopping=True)
        emoji_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # emoji_output = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

        # Check if the output is valid
        if emoji_output.strip() == "":
            raise ValueError("Empty model output.")

        return emoji_output
    except Exception:
        # Fallback logic: Match words to the emoji dataset
        words = text.lower().split()
        matched_emojis = []

        for word in words:
            # Find the first matching emoji for the word
            emoji_match = combined_emoji_df[combined_emoji_df['label'].str.contains(word, na=False, case=False)]['emoji']
            if not emoji_match.empty:
                matched_emojis.append(emoji_match.values[0])  # Get the first emoji only

        if matched_emojis:
            return " ".join(matched_emojis)
        else:
            return "No matching emoji found."

# Interactive loop with fallback
def interactive_emoji_translator_with_fallback():
    print("Welcome to the Enhanced Emoji Translator!")
    print("Type any phrase and press Enter to get an emoji.")
    print("Type 'exit' to quit the program.")

    while True:
        # Get user input
        phrase = input("Enter a phrase: ")

        # Exit condition
        if phrase.lower() == 'exit':
            print("Exiting the Emoji Translator. Goodbye!")
            break

        # Translate the phrase to an emoji
        try:
            emoji_output = translate_to_emoji_with_fallback(phrase)
            # for idx, emoji_output in enumerate(emoji_output): print(f"Output {idx+1}: {emoji_output}")
            print(f"Emoji: {emoji_output}")
        except Exception as e:
            print(f"Error: Could not translate the phrase. {e}")

# Run the interactive translator with fallback
interactive_emoji_translator_with_fallback()

Welcome to the Enhanced Emoji Translator!
Type any phrase and press Enter to get an emoji.
Type 'exit' to quit the program.
Enter a phrase: car
Emoji: 🪧
Enter a phrase: horse
Emoji: 🐎
Enter a phrase: automobile
Emoji: 🚗
Enter a phrase: truck
Emoji: 🤩
Enter a phrase: truck stop
Emoji: 🤩 ⏱
Enter a phrase: bus stop
Emoji: 🚐 ⏱
Enter a phrase: exit
Exiting the Emoji Translator. Goodbye!
